# Исследование надёжности заёмщиков

Имеется информация о заемщиках банка, включающая в себя следующие данные: 
- количество детей в семье
- общий трудовой стаж в днях
- возраст клиента в годах
- уровень образования клиента
- идентификатор уровня образования
- семейное положение
- идентификатор семейного положения
- пол клиента
- тип занятости
- имел ли задолженность по возврату кредитов
- ежемесячный доход
- цель получения кредита 

Требуется проанализировать данные и выявить факторы, наиболее влияющие на возврат кредита в срок, а также "нарисовать" портрет типичного хорошего заемщика.

## Изучение данных 

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
import numpy as np
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Предобработка данных

In [2]:
# переведем отрицательные значения трудового стажа в положительные

data['days_employed'] = data['days_employed'].apply(abs) 

In [3]:
# переведем все данные об образовании в нижний регистр

data['education'] = data['education'].str.lower()

In [4]:
# проверим на наличие дубликатов

data.duplicated().sum()

71

In [5]:
# посмотрим внимательнее на дубликаты

data[data.duplicated() == True] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для семьи
3290,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,сыграть свадьбу
4182,1,NaN,34,высшее,0,гражданский брак,1,F,сотрудник,0,NaN,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,дополнительное образование
21032,0,NaN,60,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,заняться образованием
21132,0,NaN,47,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,ремонт жилью
21281,1,NaN,30,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,покупка коммерческой недвижимости


In [6]:
# проверим, есть ли дубликаты с заполненным столбцом total_income

data[(data.duplicated())&(~data['total_income'].isna())] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


В таблице нашелся 71 дубликат, все дубликаты имеют незаполненные значения в столбце с ежемесячным доходом. Так как нет уникального идентификатора для каждого заемщика, то нельзя утверждать, что найденные дубликаты являются действительно дубликатами. Однако, впоследствии они будут удалены из-за пропусков в столбце с ежемесячным доходом.

In [7]:
# удалим пропуски в столбце с ежемесячным доходом

data.dropna(subset = ['total_income'], inplace = True)

In [8]:
# далее изучим каждый столбец по очереди

data.describe().style.format('{:.2f}')

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.00,19351.00,19351.00,19351.00,19351.00,19351.00,19351.00
mean,0.54,66914.73,43.26,0.82,0.97,0.08,167422.30
std,1.37,139030.88,12.58,0.55,1.42,0.27,102971.57
min,-1.00,24.14,0.00,0.00,0.00,0.00,20667.26
25%,0.00,927.01,33.00,1.00,0.00,0.00,103053.15
50%,0.00,2194.22,42.00,1.00,0.00,0.00,145017.94
75%,1.00,5537.88,53.00,1.00,1.00,0.00,203435.07
max,20.00,401755.40,75.00,4.00,4.00,1.00,2265604.03


In [9]:
# рассмотрим столбец с количеством детей

data['children'] = data['children'].apply(abs) # переведем отрицательное количество детей в положительное
data['children'].describe(np.arange(0,1,0.05))

count    19351.000000
mean         0.541936
std          1.369617
min          0.000000
0%           0.000000
5%           0.000000
10%          0.000000
15%          0.000000
20%          0.000000
25%          0.000000
30%          0.000000
35%          0.000000
40%          0.000000
45%          0.000000
50%          0.000000
55%          0.000000
60%          0.000000
65%          0.000000
70%          1.000000
75%          1.000000
80%          1.000000
85%          1.000000
90%          2.000000
95%          2.000000
max         20.000000
Name: children, dtype: float64

Заемщиков с тремя и более детьми не больше 5%. Максимум - заемщики с 20 детьми, что маловероятно. 65% заемщиков не имеют детей.

In [10]:
# изучим столбец с трудовым стажем
# проверим случаи, когда количество лет трудового стажа превышает возраст заемщика, при условии ненулевого возраста заемщика

display(data[(data['days_employed']/365 > data['dob_years'])&(data['dob_years'] != 0)].count())

# посмотрим на распределение этих неправильных случаев

display(data[(data['days_employed']/365 > data['dob_years'])&(data['dob_years'] != 0)]['days_employed'].describe())

# посчитаем количество заемщиков, трудившихся более 50 лет

display(data[data['days_employed'] > 18250].count()) 

# посчитаем количество заемщиков, трудившихся более 100 лет

display(data[data['days_employed'] > 36500].count()) 

children            3428
days_employed       3428
dob_years           3428
education           3428
education_id        3428
family_status       3428
family_status_id    3428
gender              3428
income_type         3428
debt                3428
total_income        3428
purpose             3428
dtype: int64

count      3428.000000
mean     365016.543146
std       21060.402744
min      328728.720605
25%      346685.613113
50%      365176.336775
75%      383256.213070
max      401755.400475
Name: days_employed, dtype: float64

children            3446
days_employed       3446
dob_years           3446
education           3446
education_id        3446
family_status       3446
family_status_id    3446
gender              3446
income_type         3446
debt                3446
total_income        3446
purpose             3446
dtype: int64

children            3445
days_employed       3445
dob_years           3445
education           3445
education_id        3445
family_status       3445
family_status_id    3445
gender              3445
income_type         3445
debt                3445
total_income        3445
purpose             3445
dtype: int64

Получим 3428 случаев, когда количество лет трудового стажа превышает возраст заемщика, при условии ненулевого возраста заемщика. Во всех этих случаях трудовой стаж превышает 328 тысяч дней, то есть является абсолютно неправдоподобным. Практически 16% данных о трудовом стаже неверны. Плюс около 10% пропусков, итого около 25% данных о трудовом стаже неизвестны.

Количество заемщиков с трудовым стажем более 50 лет - 3446, более 100 лет - 3445. 

Медианный стаж заемщиков - около 6 лет. При медианном возрасте в 42 года медианный стаж тоже выглядит странно. Однако за неимением других данных заменим пропуски в трудовом стаже медианой.

In [11]:
# заменим пропуски в столбце с трудовым стажем медианным значением

data.loc[data['days_employed'].isna(), 'days_employed'] = data['days_employed'].median()
data['days_employed'].isna().sum() # проверим, что пропуски заменились

0

In [12]:
# рассмотрим столбец с возрастом. посчитаем количество нулевых значений 

data[data['dob_years'] ==0].count()

children            91
days_employed       91
dob_years           91
education           91
education_id        91
family_status       91
family_status_id    91
gender              91
income_type         91
debt                91
total_income        91
purpose             91
dtype: int64

In [13]:
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years']

Есть 91 заемщик с нулевым возрастом, заменили нули на средний возраст.

In [14]:
# проверим столбец с образованием

display(data['education'].value_counts())
display(data['education_id'].value_counts())

среднее                13693
высшее                  4716
неоконченное высшее      675
начальное                261
ученая степень             6
Name: education, dtype: int64

1    13693
0     4716
2      675
3      261
4        6
Name: education_id, dtype: int64

В столбцах с образованием данные сходятся между собой, около 70% заемщиков имеют среднее образование.

In [15]:
# проверим столбец с семейным положением

display(data['family_status'].value_counts())
display(data['family_status_id'].value_counts())

женат / замужем          11143
гражданский брак          3735
Не женат / не замужем     2525
в разводе                 1083
вдовец / вдова             865
Name: family_status, dtype: int64

0    11143
1     3735
4     2525
3     1083
2      865
Name: family_status_id, dtype: int64

Данные о семейном положении так же сходятся между собой, чуть больше половины заемщиков 'женаты/замужем'

In [16]:
# изучим столбец с данными о поле заемщика

data['gender'].value_counts()

F      12752
M       6598
XNA        1
Name: gender, dtype: int64

In [17]:
# заменим непонятный пол на женский, так как женщин почти в два раза больше, чем мужчин

data.loc[data['gender'] == 'XNA', 'gender'] = 'F'
data['gender'].value_counts() # проверили

F    12753
M     6598
Name: gender, dtype: int64

In [18]:
# столбец с видом занятости

data['income_type'].value_counts()

сотрудник          10014
компаньон           4577
пенсионер           3443
госслужащий         1312
безработный            2
студент                1
предприниматель        1
в декрете              1
Name: income_type, dtype: int64

In [19]:
# столбец с информацией о проблемах с задолженностью

data['debt'].value_counts()

0    17780
1     1571
Name: debt, dtype: int64

In [20]:
# столбец с целью кредита, здесь много вариантов одного и того же, потом надо разбить на недвижимость, образование,
# автомобиль, свадьба, ремонт

data['purpose'].value_counts()

свадьба                                   721
сыграть свадьбу                           693
на проведение свадьбы                     685
операции с недвижимостью                  615
покупка коммерческой недвижимости         597
покупка жилья                             595
покупка жилья для сдачи                   588
жилье                                     587
операции с коммерческой недвижимостью     581
строительство жилой недвижимости          580
операции с жильем                         579
покупка своего жилья                      574
недвижимость                              572
покупка жилья для семьи                   570
строительство недвижимости                561
строительство собственной недвижимости    560
операции со своей недвижимостью           559
покупка недвижимости                      552
покупка жилой недвижимости                546
ремонт жилью                              542
автомобиль                                454
на покупку своего автомобиля      

ИТОГО:

По столбцу "дети": имелись отрицательные значения - заменили их на положительные; есть данные с очень большим количеством детей, но их немного, стоит ли их заменять на более правдоподобные - зависит от цели исследования, в данный момент не заменяем.

По столбцу "трудовой стаж в днях": отрицательные значения заменили на положительные; 16% просто неверных значений, 
чем их качественно заменить - непонятно, в идеале надо обратиться к поставщику данных. Пропуски (их было примерно 10%) заменили на медиану, хоть это и плохо. Итого суммарно 26% неверных данных.

По столбцу "возраст заемщика": были нулевые пропуски, их было немного, заменили на среднее значение возраста ненулевых.
    
Столбец "образование" привели к единому стандарту.

Столбцы "идентификатор уровня образования", "семейное положение", "идентификатор семейного положения", "тип занятости" и 
"идентификатор проблем с возвратом кредита" изменениям не подвергались.

В столбце "пол" заменили одно неопознанное значение на женский.

В столбце "ежемесячный доход" было около 10% пропусков, удалили пропуски. После очистки осталась 19 351 строка.

В целом по заемщикам:

    - примерно 2/3 заемщиков женщины, 1/3  - мужчины
    - средний возраст заемщиков - 42 года
    - чуть больше половины заемщиков женаты или замужем
    - около 70% заемщиков имеют среднее образование
    - примерно 2/3 заемщиков не имеют детей
    - около 8% имели проблемы с возвратом кредита

## Замена типов данных

In [21]:
data['days_employed'] = data['days_employed'].astype('int') # заменим вещественный числа на целые

data['total_income'] = data['total_income'].astype('int') # заменим вещественный числа на целые

data['dob_years'] = data['dob_years'].astype('int') # заменим вещественный числа на целые

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
children            19351 non-null int64
days_employed       19351 non-null int64
dob_years           19351 non-null int64
education           19351 non-null object
education_id        19351 non-null int64
family_status       19351 non-null object
family_status_id    19351 non-null int64
gender              19351 non-null object
income_type         19351 non-null object
debt                19351 non-null int64
total_income        19351 non-null int64
purpose             19351 non-null object
dtypes: int64(7), object(5)
memory usage: 2.5+ MB


Пропуски убрали, вещественные числа перевели в целые.

## Лемматизация

In [22]:
# посчитаем общую частотность слов в столбце в целью кредита

string = ','.join(data['purpose'])
lemmas = m.lemmatize(string)
Counter(lemmas)            

Counter({'покупка': 5353,
         ' ': 30248,
         'жилье': 4035,
         ',': 19350,
         'приобретение': 419,
         'автомобиль': 3897,
         'дополнительный': 810,
         'образование': 3597,
         'сыграть': 693,
         'свадьба': 2099,
         'операция': 2334,
         'с': 2619,
         'на': 2016,
         'проведение': 685,
         'для': 1158,
         'семья': 570,
         'недвижимость': 5723,
         'коммерческий': 1178,
         'жилой': 1126,
         'строительство': 1701,
         'собственный': 560,
         'подержать': 771,
         'свой': 2013,
         'со': 559,
         'заниматься': 797,
         'получение': 1179,
         'сделка': 844,
         'высокий': 1243,
         'профильный': 389,
         'сдача': 588,
         'ремонт': 542,
         'подержанный': 101,
         '\n': 1})

Получаем, что основные цели кредита это: свадьба, жилье/недвижимость, автомобиль, образование, ремонт.

In [23]:
# функция для выявления цели кредита

def purposes (purpose): 
    lemmas = m.lemmatize(purpose)
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование' in lemmas:
        return 'образование'
    if 'ремонт' in lemmas:
        return 'ремонт'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'жилье' or 'недвижимость' in lemmas:
        return 'недвижимость'
    return 'другое'

data['purpose_short'] = data['purpose'].apply(purposes) # добавляем новый столбец с целью кредита
data['purpose_short'].value_counts() # проверяем новый столбец               

недвижимость    9216
автомобиль      3897
образование     3597
свадьба         2099
ремонт           542
Name: purpose_short, dtype: int64

Почти половина кредитов берется на покупку жилья, по 20% на автомобиль и образование.

## Категоризация данных

In [24]:
# напишем функцию, распределяющую заемщиков по разным уровням дохода

q1 = data['total_income'].quantile(0.25)
q2 = data['total_income'].quantile(0.5)
q3 = data['total_income'].quantile(0.75)

def group_income(income): 
    if income <= q1:
        return 1 
    if income > q1 and income <= q2:
        return 2
    if income > q2 and income <= q3:
        return 3
    return 4

# применим данную функцию к столбцу с ежемесячным доходом и на его основе сформируем новый столбец

data['level_income'] = data['total_income'].apply(group_income)
display(q1,q2,q3)

103053.0

145017.0

203434.5

In [25]:
columns = ['gender','children', 'family_status_id','level_income','purpose_short']
for column in columns:
    display(data.groupby(column).agg({'debt': ['count', 'sum', 'mean']}))

debt               
        count  sum      mean
gender                      
F       12753  895  0.070180
M        6598  676  0.102455

debt               
          count  sum      mean
children                      
0         12710  952  0.074902
1          4387  409  0.093230
2          1851  177  0.095624
3           294   22  0.074830
4            34    3  0.088235
5             8    0  0.000000
20           67    8  0.119403

debt               
                  count  sum      mean
family_status_id                      
0                 11143  846  0.075922
1                  3735  339  0.090763
2                   865   56  0.064740
3                  1083   76  0.070175
4                  2525  254  0.100594

debt               
             count  sum      mean
level_income                     
1             4838  383  0.079165
2             4838  421  0.087019
3             4837  426  0.088071
4             4838  341  0.070484

debt               
              count  sum      mean
purpose_short                     
автомобиль     3897  367  0.094175
недвижимость   9216  684  0.074219
образование    3597  331  0.092021
ремонт          542   31  0.057196
свадьба        2099  158  0.075274

Большая разница по просрочкам между мужчинами и женщинами, у женщин средняя просрочка 7%, у мужчин 10,2%. 

Просрочка у бездетных заемщиков 7,5% (меньше средней по портфелю равной 8%), у заемщиков с одним-двумя детьми - больше 9%, заемщиков с тремя и  более детьми мало, поэтому по ним выводы делать не будем. В целом, можно выдвинуть гипотезу, что заемщики с детьми допускают просрочки чаще. Скорее всего это связано с тем, что при тех же зарплатах у заемщиков с детьми больше расходов, нежели у заемщиков без детей. Плюс у заемщиков с детьми могут чаще возникать непредвиденные расходы.

Просрочка у женатых/замужних заемщиков составляет 7,6%. Самые недисциплинированные заемщики это пары, живущие в гражданском браке (просрочка 9,1%), и неженатые/незамужние (просрочка 10,1%). Я бы объяснила это тем, что женатые/замужние чаще ведут общий бюджет, владеют общим имуществом, если сумма кредита была потрачена на семейные нужды, то при просрочке банк может обратиться в суд и потребовать выплаты по кредиту за счет общего имущества супругов, даже если второй супруг не был созаемщиком или поручителем. При этом разведенные и вдовствующие имеют небольшую просрочку от 6,5 до 7%.

У заемщиков с доходом более 203 тысяч рублей просрочка составляет 7%, у заемщиков с доходом от 103 до 203 тыс. просрочка 8,7-8,8% (это половина всех заемщиков), у заемщиков с доходом до 103 тысяч просрочка 7,9%. 

Заемщики, берущие кредит на покупку автомобиля или образование, допускают больше просрочек (более 9%), нежели остальные заемщики. С кредитами на покупку автомобиля надо бы посмотреть, отличается ли уровень просрочки у кредитов с залогом и беззалоговых кредитов. Подозреваю, что заемщики, у которых купленный автомобиль находится в залоге у банка, платят более дисциплинированно.

Заемщики, берущие кредит на ремонт, допускают низкую просрочку (5,7%), но их мало, поэтому о них сложно делать выводы.

Просрочка кредитов, взятых на свадьбу, составляет 7,5%. Удивительно, что просрочка у свадеб меньше, чем просрочка у кредитов на образование. И там, и там суммы кредитов скорее всего не очень большие (по сравнению с кредитами на автомобили и недвижимость), но тем не менее молодожены платят лучше. Можно это попытаться объяснить там, что молодожены получают подарки на свадьбу деньгами, которые помогают отбить стоимость самой свадьбы. А в случае с образованием возможно заемщики рассчитывают на повышение дохода после получения образования, что может и не происходить или происходить не сразу.

По кредитам, взятым на покупку недвижимости, просрочка допускается в 7,4% случаев. Просрочку ниже средней по покупке недвижимости можно объяснить тем, что обычно такие кредиты сопровождаются серьезными залогами (либо это ипотека, либо другой актив в залоге).

In [26]:
# посмотрим данные о средней просрочке в разбивке по полу заемщика

for column in ['children', 'family_status_id','level_income','purpose_short']:
    display(data.groupby(['gender', column]).agg({'debt': ['count', 'sum', 'mean']}))

debt               
                count  sum      mean
gender children                     
F      0         8569  527  0.061501
       1         2800  226  0.080714
       2         1132  123  0.108657
       3          180   14  0.077778
       4           24    1  0.041667
       5            7    0  0.000000
       20          41    4  0.097561
M      0         4141  425  0.102632
       1         1587  183  0.115312
       2          719   54  0.075104
       3          114    8  0.070175
       4           10    2  0.200000
       5            1    0  0.000000
       20          26    4  0.153846

debt               
                        count  sum      mean
gender family_status_id                     
F      0                 6995  485  0.069335
       1                 2560  205  0.080078
       2                  815   46  0.056442
       3                  843   54  0.064057
       4                 1540  105  0.068182
M      0                 4148  361  0.087030
       1                 1175  134  0.114043
       2                   50   10  0.200000
       3                  240   22  0.091667
       4                  985  149  0.151269

debt               
                    count  sum      mean
gender level_income                     
F      1             3830  263  0.068668
       2             3391  267  0.078738
       3             2945  208  0.070628
       4             2587  157  0.060688
M      1             1008  120  0.119048
       2             1447  154  0.106427
       3             1892  218  0.115222
       4             2251  184  0.081741

debt               
                     count  sum      mean
gender purpose_short                     
F      автомобиль     2573  211  0.082005
       недвижимость   6044  384  0.063534
       образование    2381  186  0.078118
       ремонт          341   20  0.058651
       свадьба        1414   94  0.066478
M      автомобиль     1324  156  0.117825
       недвижимость   3172  300  0.094578
       образование    1216  145  0.119243
       ремонт          201   11  0.054726
       свадьба         685   64  0.093431

У женщин возвратность кредита ухудшают следующие факторы: наличие детей (наличие двух детей ухудшает сильнее, чем наличие одного ребенка), гражданский брак (семейные и незамужние женщины имеют примерно одинаковый уровень просрочки 6,8-6,9%), цель кредита "автомобиль" или "образование".  

У мужчин самые низкие проценты просрочки у женатых (8,7%), имеющих ровно двух детей (7,5%), зарабатывающих более 203 тысяч рублей в месяц (8,2%). Самые высокие проценты просрочки у мужчин, которые неженаты - 15,1%, с целью кредита на "автомобиль" или "образование" - 11,8-11,9%, зарабатывающих меньше 103 тысяч рублей в месяц - 11,9%. 

## Вывод

>Итого, наибольшая разница по просрочкам наблюдается между мужчинами и женщинами.

>Наиболее привлекательными для банка заемщиками являются бездетные женщины, не состоящие в гражданском браке, с целями кредита не "автомобиль" и не "образование". У женщин-заемщиков гражданский брак, наличие одного ребенка или цели "автомобиль"/"образование" ухудшают возврат кредита, приближая просрочку к средней по всему портфелю (8%). Наличие двух и более детей у женщин-заемщиков еще больше ухудшает возврат кредита. У женщин-заемщиков с уровнем дохода до 203 тысяч просрочка составляет около 6,9-7,9%, свыше 203 тысяч - 6,1%.

>Наиболее рискованными заемщиками являются неженатые мужчины. Их уровень просрочки составляет 15,1%. По заемщикам-мужчинам можно отметить, что существенное улучшение возрата кредита происходит, если мужчина женат, наличие двух детей или наличие ежемесячного дохода более 203 тысяч рублей. Так же можно отметить, что наличие одного ребенка у мужчины отрицательно влияет на возврат кредита. Так же отрицательно влияют на возврат кредита цели "покупка автомобиля" или "получение образования". 